In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, num_layers, cell_type='RNN'):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        if cell_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.rnn(x, (h0, c0))
        
        out = self.fc(out[:, -1, :]) 
        return out

In [4]:
# # Fully connected neural network with one hidden layer
# class RNN(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(RNN, self).__init__()
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         # -> x needs to be: (batch_size, seq, input_size)
        
#         # or:
#         #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#         #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, num_classes)
        
#     def forward(self, x):
#         # Set initial hidden states (and cell states for LSTM)
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
#         #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        
#         # x: (n, 28, 28), h0: (2, n, 128)
        
#         # Forward propagate RNN
#         out, _ = self.rnn(x, h0)
#         print(out)
#         print(_)
#         # or:
#         #out, _ = self.lstm(x, (h0,c0))  
        
#         # out: tensor of shape (batch_size, seq_length, hidden_size)
#         # out: (n, 28, 128)
        
#         # Decode the hidden state of the last time step
#         out = out[:, -1, :]
#         # out: (n, 128)
         
#         out = self.fc(out)
#         # out: (n, 10)
#         return out

In [5]:
air_objects = [
    AirObject(position=[ 4000,  4000,  9980], velocity=[-20, -20, 0], acceleration=[0, 0, 0]),
    AirObject(position=[ 4000, -4000, 10020], velocity=[-20,  20, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-4000,  4000, 10000], velocity=[ 10, -20, 0], acceleration=[0, 0, 0]),
    AirObject(position=[-4000, -4000, 10000], velocity=[ 20,  10, 0], acceleration=[0, 0, 0])
]

radar_systems = [
    RadarSystem(position=[ 4000,  4000, 0], detection_radius=100000, error=20),
    RadarSystem(position=[ 4000, -4000, 0], detection_radius=100000, error=20),
    RadarSystem(position=[-4000,  4000, 0], detection_radius=100000, error=20),
    RadarSystem(position=[-4000, -4000, 0], detection_radius=100000, error=20)
]

air_env = AirEnv()
for ao in air_objects:
    air_env.attach_air_object(ao)

control_point = ControlPoint()
for rs in radar_systems:
    rs.attach_air_environment(air_env)
    control_point.attach_radar_system(rs)

supervisor = Supervisor()
supervisor.attach_air_environment(air_env)
supervisor.attach_control_point(control_point)
supervisor.attach_radar_systems(radar_systems)

In [6]:
dt          =   0.5
t_train_min =   0.0
t_train_max = 200.0
t_test_max  = 300.0

In [7]:
air_env.set_public_ids(True)
supervisor.run(t_train_min, t_train_max, dt)

air_env.set_public_ids(False)
supervisor.run(t_train_max + dt, t_test_max, dt)

In [8]:
data = control_point.get_data()[['detection_time', 'radar_system_id', 'air_object_id', 'detection_error',
                                 'air_object_x', 'air_object_y', 'air_object_z']]
data

,detection_time,radar_system_id,air_object_id,detection_error,air_object_x,air_object_y,air_object_z
0,0.0,0.0,0,20,3974.925331,4029.571578,9984.310764
1,0.0,0.0,1,20,3996.767680,-4016.421875,10019.599076
2,0.0,0.0,2,20,-4005.442061,4027.112450,10017.857596
3,0.0,0.0,3,20,-3999.387330,-4020.318595,9976.279052
4,0.0,1.0,0,20,4025.577859,3998.637210,9980.620848
...,...,...,...,...,...,...,...
9611,300.0,2.0,-1,20,1995.613124,-1023.387732,9951.305349
9612,300.0,3.0,-1,20,-1974.494439,-1988.505636,10004.685671
9613,300.0,3.0,-1,20,-1987.503979,1993.180194,10064.460175
9614,300.0,3.0,-1,20,-997.198242,-1987.543967,9992.141455


In [9]:
x_train = data[(data['detection_time'] >= t_train_min) & (data['detection_time'] <= t_train_max) & (data['air_object_id'].isna() >= 0)][[
    'detection_time', 'radar_system_id', 'air_object_x', 'air_object_y', 'air_object_z', 'detection_error'
]]
y_train = data[(data['detection_time'] >= t_train_min) & (data['detection_time'] <= t_train_max) & (data['air_object_id'].isna() >= 0)][
    'air_object_id'
]

print(f'Сформирована выборка для обучения')
print(f'Размер датафрейма признаков - {x_train.shape}')
print(f'Размер датафрейма ответов - {y_train.shape}')

Сформирована выборка для обучения
Размер датафрейма признаков - (6416, 6)
Размер датафрейма ответов - (6416,)


In [10]:
x_test = data[(data['detection_time'] > t_train_max) & (data['detection_time'] <= t_test_max) & (data['air_object_id'].isna() == -1)][[
    'detection_time', 'radar_system_id', 'air_object_x', 'air_object_y', 'air_object_z', 'detection_error'
]]

print(f'Сформирована выборка для обучения')
print(f'Размер датафрейма признаков - {x_test.shape}')

Сформирована выборка для обучения
Размер датафрейма признаков - (0, 6)


In [13]:
train = []
for t in np.linspace(t_train_min, t_train_max, int((t_train_max - t_train_min)/dt + 1)):
    train_t_df = data[data['detection_time'] == t][['air_object_x', 'air_object_y', 'air_object_z', 'detection_error', 'air_object_id']]
    # print(train_t_df.dtypes)
    # print(xt_train.shape)
    train_t = train_t_df.to_numpy()
    # print(xt_train)
    train.append(train_t)

train = torch.from_numpy(np.array(train)).transpose(0, 1)
print(train.shape)

torch.Size([16, 401, 5])


In [14]:
test = []
for t in np.linspace(t_train_max + dt, t_test_max, int((t_test_max - t_train_max)/dt)):
    test_t_df = data[data['detection_time'] == t][['air_object_x', 'air_object_y', 'air_object_z', 'detection_error', 'air_object_id']]
    # print(train_t_df.dtypes)
    # print(xt_train.shape)
    test_t = test_t_df.to_numpy()
    # print(xt_train)
    test.append(test_t)

test = torch.from_numpy(np.array(test)).transpose(0, 1)
print(test.shape)

torch.Size([16, 200, 5])


In [12]:
raise RuntimeError()

RuntimeError: 

In [ ]:
# num_classes = 10
# num_epochs = 2
batch_size = 16
learning_rate = 0.001

input_size = 4
sequence_length = int((t_train_max - t_train_min)/dt + 1)
# hidden_size = 128
# num_layers = 2

In [ ]:
model = RNN(
    input_size=input_size,
    hidden_size=6,
    output_size=1,
    num_layers=2,
    cell_type='LSTM'
).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  

In [ ]:
# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')